In [38]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from tensorflow.keras.optimizers.legacy import Adam
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from sklearn.model_selection import train_test_split

In [39]:
df = pd.read_csv('Bank_Customers.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [40]:
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [41]:
le = LabelEncoder()

In [42]:
le.fit(df['Gender'])
e = le.transform(df['Gender'])
e

array([0, 0, 0, ..., 0, 1, 0])

In [43]:
df.drop(columns=['Gender'], inplace=True)
df['Gender'] = e

In [44]:
df

,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender
0,619,France,42,2,0.00,1,1,1,101348.88,1,0
1,608,Spain,41,1,83807.86,1,0,1,112542.58,0,0
2,502,France,42,8,159660.80,3,1,0,113931.57,1,0
3,699,France,39,1,0.00,2,0,0,93826.63,0,0
4,850,Spain,43,2,125510.82,1,1,1,79084.10,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,39,5,0.00,2,1,0,96270.64,0,1
9996,516,France,35,10,57369.61,1,1,1,101699.77,0,1
9997,709,France,36,7,0.00,1,0,1,42085.58,1,0
9998,772,Germany,42,3,75075.31,2,1,0,92888.52,1,1


In [45]:
enc = OneHotEncoder()
X=df['Geography']

In [46]:
ohe = OneHotEncoder()
transformed = ohe.fit_transform(df[['Geography']])
k= pd.DataFrame(transformed.toarray())
df = pd.concat([df,k], axis=1)
df.drop(columns=['Geography'], inplace=True)

In [47]:
df.rename(columns = {0:'g1', 1:'g2', 2:'g3'}, inplace = True)

In [48]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender,g1,g2,g3
0,619,42,2,0.00,1,1,1,101348.88,1,0,1.0,0.0,0.0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0.0,0.0,1.0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,1.0,0.0,0.0
3,699,39,1,0.00,2,0,0,93826.63,0,0,1.0,0.0,0.0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0.0,0.0,1.0


In [49]:
X = df.drop(columns=['Exited'])
y = df['Exited']

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size = .20)

In [51]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [52]:
ann = Sequential()

In [53]:
ann.add(Dense(units =6 , activation = "relu"))
ann.add(Dense(units =6 , activation = "relu"))
ann.add(Dense(units =1 , activation = "sigmoid"))

In [54]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [55]:
ann.fit(X_train,y_train, batch_size =20, epochs = 40 )

Epoch 1/40
400/400 [==============================] - 1s 3ms/step - loss: 0.5575 - accuracy: 0.7244
Epoch 2/40
400/400 [==============================] - 1s 3ms/step - loss: 0.4439 - accuracy: 0.8013
Epoch 3/40
400/400 [==============================] - 1s 3ms/step - loss: 0.4251 - accuracy: 0.8119
Epoch 4/40
400/400 [==============================] - 1s 3ms/step - loss: 0.4102 - accuracy: 0.8226
Epoch 5/40
400/400 [==============================] - 1s 3ms/step - loss: 0.3990 - accuracy: 0.8268
Epoch 6/40
400/400 [==============================] - 1s 3ms/step - loss: 0.3912 - accuracy: 0.8269
Epoch 7/40
400/400 [==============================] - 1s 3ms/step - loss: 0.3841 - accuracy: 0.8329
Epoch 8/40
400/400 [==============================] - 1s 3ms/step - loss: 0.3744 - accuracy: 0.8394
Epoch 9/40
400/400 [==============================] - 1s 3ms/step - loss: 0.3649 - accuracy: 0.8453
Epoch 10/40
400/400 [==============================] - 1s 3ms/step - loss: 0.3565 - accuracy: 0.8505

In [56]:
test  = ann.predict(sc.transform([[502, 1, 42, 1, 80000, 1, 1, 1, 100000, 0 ,1, 0]]))
test

1/1 [==============================] - 0s 43ms/step


/Users/amjaadb/miniconda3/envs/mlp/lib/python3.8/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[1.]], dtype=float32)